In [1]:
import numpy as np
import scipy.io as spio
from matplotlib import pyplot as plt
from matplotlib import cm
import glob
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
import numpy as np
from util.models import MODELS
from util.tasks import TASKS
import tensorflow_probability as tfp

from tfds_data.aff_nist import AffNist
from calibration_util import *
from notebook_utils import *
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns; sns.set()
sns.set_style("whitegrid")

from tqdm import tqdm

def visualization(x, y,count,index):
    x = np.reshape(x, (40, 40))
    
    plt.subplot(1,count,index)
    plt.imshow(x, cmap=cm.Greys_r)
    plt.title(y)
    plt.axis('off')   

def visualization_overlap(x0,x1, y0,y1,count,index):
    r = np.reshape(x0, (40, 40))
    g = np.reshape(x1, (40, 40))
    b = np.zeros_like(r)
    rgb = np.stack([r,g,b],-1)
    
    plt.subplot(1,count,index)
    plt.imshow(rgb)
    plt.title('R:('+ str(y0)+','+str(y1)+')')
    plt.axis('off')   


[nltk_data] Downloading package punkt to /home/dehghani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
chkpt_dir='../tf_ckpts'
task_name = 'mnist40'
mnist = TASKS[task_name](get_task_params(), data_dir='../data')

chkpt_dir='../tf_ckpts'
task_name = 'affnist'
affnist = TASKS[task_name](get_task_params(), data_dir='../data')

In [ ]:
count = 10
for x,y in affnist.test_dataset:
    print(x.shape)
    #visualization(x[1].numpy(),y[1].numpy(),1,1)
    plt.imshow(x[0,...,0])
    plt.show()
    count -= 1
    if count <= 0:
        break

In [ ]:
count = 10
for x,y in mnist.valid_dataset:
    print(x.shape)
    #visualization(x[1].numpy(),y[1].numpy(),1,1)
    plt.imshow(x[0,...,0])
    plt.show()
    count -= 1
    if count <= 0:
        break

In [13]:
config={'exp_name':'test',
    'model_config':'ff_mnist2',
    'task_name':'mnist40',
    'model_name':'cl_vff',
    'chkpt_dir':'../tf_ckpts',
    'learning_rate': 0.001
    }

task = TASKS[config['task_name']](get_task_params(batch_size=16), data_dir='../data')

hparams = get_model_params(task, config['model_name'], config['model_config'])
print(hparams)

model, _ = get_model(config, task, hparams, 0)

model config: ff_mnist2
{}
No checkpoint found ../tf_ckpts/mnist40/cl_vff_h-1024_d-3_hdrop-0.5_indrop-0.2_ff_mnist2_0.001_test


In [21]:
class VanillaFF(tf.keras.models.Sequential):
  def __init__(self, hparams, scope="cl_vff", *inputs, **kwargs):
    if 'cl_token' in kwargs:
      del kwargs['cl_token']

    super(VanillaFF, self).__init__()
    self.scope = scope
    self.hparams = hparams

    self.model_name = '_'.join([self.scope,
                                'h-' + str(self.hparams.hidden_dim),
                                'd-' + str(self.hparams.depth),
                                'hdrop-' + str(self.hparams.hidden_dropout_rate),
                                'indrop-' + str(self.hparams.input_dropout_rate)])

    self.regularizer = tf.keras.regularizers.l1_l2(l1=0.00,
                                                   l2=0.00001)
    self.create_vars()
    self.rep_index = 1
    self.rep_layer = -1



  def create_vars(self):
    #self.batch_norm0 = tf.keras.layers.BatchNormalization()
    self.conv1 = tf.keras.layers.Conv2D(32, (3,3),
                                  activation=None,
                                  kernel_regularizer=self.regularizer)
    self.flat = tf.keras.layers.Flatten()
    self.batch_norm = tf.keras.layers.BatchNormalization()
    self.batch_norm.trainable = True
    self.indrop = tf.keras.layers.Dropout(self.hparams.input_dropout_rate)

    self.hidden_layers = []
    self.hidden_batch_norms = []
    self.hidden_dropouts = []
    for i in np.arange(2):
      self.hidden_layers.append(tf.keras.layers.Dense(self.hparams.hidden_dim,
                                     activation='relu',
                                     kernel_regularizer=self.regularizer))
      self.hidden_batch_norms.append(tf.keras.layers.BatchNormalization())
      self.hidden_batch_norms[i].trainable = True
      self.hidden_dropouts.append(tf.keras.layers.Dropout(self.hparams.hidden_dropout_rate))

    self.final_dense = tf.keras.layers.Dense(self.hparams.output_dim,
                                   kernel_regularizer=self.regularizer)


  def call(self, inputs, padding_symbol=None, training=None, **kwargs):
    x = inputs # self.batch_norm0(inputs, training=training, **kwargs)
    x = self.conv1(x, training=training, **kwargs)
    x = self.flat(x, **kwargs)
    x = self.batch_norm(x, training=training, **kwargs)
    x = self.indrop(x, training=training, **kwargs)

    for i in np.arange(2):
      x = self.hidden_layers[i](x, training=training, **kwargs)
      x = self.hidden_batch_norms[i](x, training=training, **kwargs)
      x = self.hidden_dropouts[i](x, training=training, **kwargs)

    logits = self.final_dense(x, training=training, **kwargs)

    return logits


  def detailed_call(self, inputs, padding_symbol=None, training=None, **kwargs):
    layer_activations = []
    x = self.flat(inputs, **kwargs)
    x = self.batch_norm(x, training=training, **kwargs)
    x = self.indrop(x, training=None, **kwargs)
    layer_activations.append(x)

    for i in np.arange(self.hparams.depth):
      x = self.hidden_layers[i](x, training=None, **kwargs)
      x = self.hidden_batch_norms[i](x, training=None, **kwargs)
      x = self.hidden_dropouts[i](x, training=None, **kwargs)
      layer_activations.append(x)

    pnltimt = x
    logits = self.final_dense(x, training=None, **kwargs)

    return logits, pnltimt, layer_activations


In [22]:
model = VanillaFF(hparams)
for x,y in task.train_dataset:
    print(x.shape, y.shape)
    break
model(inputs=x, training=True)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=task.get_loss_fn(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.summary()

(16, 40, 40, 1) (16,)
Model: "vanilla_ff_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_34 (Batc multiple                  4         
_________________________________________________________________
conv2d_16 (Conv2D)           multiple                  320       
_________________________________________________________________
flatten_5 (Flatten)          multiple                  0         
_________________________________________________________________
batch_normalization_35 (Batc multiple                  184832    
_________________________________________________________________
dropout_20 (Dropout)         multiple                  0         
_________________________________________________________________
dense_22 (Dense)             multiple                  47318016  
_________________________________________________________________
dense_23 (Dense)             mul

In [23]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=task.get_loss_fn(),
              metrics=task.metrics())

history = model.fit(task.train_dataset,
                  epochs=10,
                  steps_per_epoch=task.n_train_batches,
                  validation_steps=task.n_valid_batches,
                  validation_data=task.valid_dataset,
                  verbose=2
                  )



Train for 3750 steps, validate for 625 steps
Epoch 1/10
3750/3750 - 39s - loss: 0.5405 - classification_loss: 0.4357 - sparse_categorical_accuracy: 0.8764 - val_loss: 0.3278 - val_classification_loss: 0.1726 - val_sparse_categorical_accuracy: 0.9481
Epoch 2/10
3750/3750 - 37s - loss: 0.5107 - classification_loss: 0.2876 - sparse_categorical_accuracy: 0.9155 - val_loss: 0.4100 - val_classification_loss: 0.1192 - val_sparse_categorical_accuracy: 0.9646
Epoch 3/10
3750/3750 - 37s - loss: 0.6335 - classification_loss: 0.2704 - sparse_categorical_accuracy: 0.9201 - val_loss: 87.6646 - val_classification_loss: 87.0887 - val_sparse_categorical_accuracy: 0.1137
Epoch 4/10
3750/3750 - 37s - loss: 0.7220 - classification_loss: 0.2560 - sparse_categorical_accuracy: 0.9234 - val_loss: 0.5119 - val_classification_loss: 0.1056 - val_sparse_categorical_accuracy: 0.9686
Epoch 5/10
3750/3750 - 37s - loss: 0.6356 - classification_loss: 0.2276 - sparse_categorical_accuracy: 0.9320 - val_loss: 0.7172 - va

In [24]:
model.evaluate(affnist.test_dataset, steps=affnist.n_test_batches)

5000/5000 [==============================] - 20s 4ms/step - loss: 35.1139 - classification_loss: 34.7236 - sparse_categorical_accuracy: 0.1398


[35.11385690574646, 34.7236, 0.13975625]